<h1>I/O Tools - Matplotlib</h1><h2 align="center">Multiple Elements in a Single Chart</h2>

<h3>Multiple-Line Charts</h3>

In [ ]:
from matplotlib.pyplot import gca, figure, show
from matplotlib.axes import Axes
from pandas import read_csv, DataFrame, Series
from dslabs_functions import set_chart_labels, set_chart_xticks


def plot_multiline_chart(
    xvalues: list,
    yvalues: dict,
    ax: Axes = None,  # type: ignore
    title: str = "",
    xlabel: str = "",
    ylabel: str = "",
    percentage: bool = False,
) -> Axes:
    if ax is None:
        ax = gca()
    ax = set_chart_labels(ax=ax, title=title, xlabel=xlabel, ylabel=ylabel)
    ax = set_chart_xticks(xvalues, ax=ax, percentage=percentage)
    legend: list = []
    for name, y in yvalues.items():
        ax.plot(xvalues, y)
        legend.append(name)
    ax.legend(legend, fontsize="xx-small")
    return ax


data: DataFrame = read_csv("data/algae.csv", index_col="date")
two_series: dict[str, Series] = {
    "Phosphate": data["Phosphate"],
    "Orthophosphate": data["Orthophosphate"],
}
figure(figsize=(12, 4))
plot_multiline_chart(
    data.index.to_list(),
    two_series,
    title="Phosphate and Orthophosphate values",
    xlabel="date",
    ylabel="values",
)
show()

<h3>Multiple-Bar Charts</h3>

In [ ]:
from numpy import arange, ndarray
from matplotlib.container import BarContainer
from dslabs_functions import FONT_TEXT


def plot_multibar_chart(
    group_labels: list,
    yvalues: dict,
    ax: Axes = None,  # type: ignore
    title: str = "",
    xlabel: str = "",
    ylabel: str = "",
    percentage: bool = False,
) -> Axes | list[Axes]:
    if ax is None:
        ax = gca()
    ax = set_chart_labels(ax=ax, title=title, xlabel=xlabel, ylabel=ylabel)
    if percentage:
        ax.set_ylim(0.0, 1.0)
    bar_labels: list = list(yvalues.keys())

    # This is the location for each bar
    index: ndarray = arange(len(group_labels))
    bar_width: float = 0.8 / len(bar_labels)
    ax.set_xticks(index + bar_width / 2, labels=group_labels)

    for i in range(len(bar_labels)):
        values: BarContainer = ax.bar(
            index + i * bar_width,
            yvalues[bar_labels[i]],
            width=bar_width,
            label=bar_labels[i],
        )
        format = "%.2f" if percentage else "%.0f"
        ax.bar_label(values, fmt=format, fontproperties=FONT_TEXT)
    ax.legend(fontsize="xx-small")
    return ax


two_series: dict[str, Series] = {
    "river_depth": data["river_depth"].value_counts().sort_index(),
    "fluid_velocity": data["fluid_velocity"].value_counts().sort_index(),
}
figure()
plot_multibar_chart(
    ["high", "low", "medium"],
    two_series,
    title="Frequency for some variables",
    ylabel="frequency",
)
show()

<h3>Multiple Scatter Charts</h3>

In [ ]:
from pandas import DataFrame
from pandas.api.types import is_integer_dtype, is_any_real_numeric_dtype
from matplotlib.collections import PathCollection
from matplotlib.colorbar import Colorbar
from matplotlib.pyplot import gcf
from dslabs_functions import ACTIVE_COLORS, FILL_COLOR


def plot_multi_scatters_chart(
    data: DataFrame, var1: str, var2: str, var3: str = "", ax: Axes = None  # type: ignore
) -> Axes:
    if ax is None:
        ax = gca()

    title: str = f"{var1} x {var2}"
    if var3 != "":
        title += f"per {var3}"
        if is_any_real_numeric_dtype(data[var3]) and not is_integer_dtype(data[var3]):
            chart: PathCollection = ax.scatter(
                data[var1], data[var2], c=data[var3].to_list()
            )
            cbar: Colorbar = gcf().colorbar(chart)
            cbar.outline.set_visible(False)  # type: ignore
            cbar.set_label(var3, loc="top")
        else:
            values: list = data[var3].unique().tolist()
            values.sort()
            for i in range(len(values)):
                subset: DataFrame = data[data[var3] == values[i]]
                ax.scatter(
                    subset[var1], subset[var2], color=ACTIVE_COLORS[i], label=values[i]
                )
            ax.legend(fontsize="xx-small")
    else:
        ax.scatter(data[var1], data[var2], color=FILL_COLOR)
    ax = set_chart_labels(ax=ax, title=title, xlabel=var1, ylabel=var2)
    return ax

In [ ]:
var1 = "Phosphate"
var2 = "Orthophosphate"
var3 = "season"

figure()
plot_multi_scatters_chart(data, var1, var2, var3)
show()

In [ ]:
var3 = "pH"
figure()
plot_multi_scatters_chart(data, var1, var2, var3)
show()